In [1]:
!pip install torch pandas numpy simpletransformers nltk faiss-cpu

In [2]:
import json
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from simpletransformers.retrieval import RetrievalModel, RetrievalArgs
from nltk import sent_tokenize
import os
import sqlite3
import torch

torch.cuda.is_available()


True

In [3]:

db_path = '../data/wiki_data_idxed.sqlite'
ir_train_tsv_path = '../data/ir_wiki_train_qanta_dev_summary.tsv'

con = sqlite3.connect(db_path)
cur = con.cursor()

In [20]:
# create database that maps title to contents of article

cur.execute('DROP TABLE IF EXISTS wiki_text')
cur.execute('CREATE TABLE wiki_text (page text, text text, text_index text PRIMARY KEY)')

seen = set()

with open('../data/wiki_text_data.jsonl', 'r', encoding='utf-8') as wiki_data:
    for line in wiki_data:
        data = json.loads(line)
    
        idx = data['text'].split('\n\n')[0].replace('\n', ' ').replace('\r', '').replace('\t', ' ')[:64]
        
        if idx == '':
            continue
        
        if idx in seen:
#             print('Seen', data['page'], data['page'] in seen_pages, idx)
#             x = cur.execute('SELECT * FROM wiki_text WHERE text_index = ?', (idx,)).fetchone()
#             print(x)
            continue
        seen.add(idx)
        
        cur.execute(f'INSERT INTO wiki_text VALUES (?, ?, ?)', (data['page'], data['text'], idx))

con.commit()

In [79]:
# create wiki summary passages
with open('../data/wiki_text_data.jsonl', 'r', encoding='utf-8') as wiki_data:
    with open('../data/wiki_summaries.txt', 'w', encoding='utf-8') as wiki_summaries:
        for line in wiki_data:
            data = json.loads(line)
            text = data['text']
            
            summary = text.split('\n\n')[0].replace('\n', ' ').replace('\r', '').replace('\t', ' ')
            wiki_summaries.write(summary + '\n')
        
        

In [91]:
# create all wiki passages
with open('../data/wiki_text_data.jsonl', 'r', encoding='utf-8') as wiki_data:
    with open('../data/wiki_summaries.tsv', 'w', encoding='utf-8') as wiki_passages:
        
        wiki_passages.write('title\tgold_passage\n')
        
        for line in wiki_data:
            data = json.loads(line)
            text = data['text']
            page = data['page']
            
            for passage in text.split('\n\n'):
                cleaned_passage = passage.replace('\n', ' ').replace('\r', '').replace('\t', ' ')
                
                if len(cleaned_passage) > 100 and page:
                    wiki_passages.write(page + '\t' + cleaned_passage + '\n')  
                break
        

In [4]:
def get_wiki_text(cur, title):
    res = cur.execute('SELECT text FROM wiki_text WHERE page = ?', (title,)).fetchone()
    if not res:
        return None
    return res[0]

In [9]:

# if not os.path.exists(ir_train_tsv_path):
with open('../data/ir_wiki_train_summaries_no_for_10_pts_qanta_train.tsv', 'w', encoding='utf-8') as ir_train_f:
    with open('../data/ir_wiki_train_summaries_no_for_10_pts_qanta_train_last_sent_only.tsv', 'w', encoding='utf-8') as ir_train_last_sent_only_f:
        ir_train_f.write('query_text\ttitle\tgold_passage\n')

        with open('../data/qanta_train.json', 'r', encoding='utf-8') as train_f:

            questions = json.load(train_f)['questions']
            print(len(questions), 'questions total')
            i = 0
            for question in questions:

                wiki_text = get_wiki_text(cur, question['page'])
                if not wiki_text:
                    print(question['page'], 'has no wiki text')
                    continue

                if not question['page']:
                    continue

                gold_passage = wiki_text.split('\n\n')[0].replace('\n', ' ').replace('\r', '').replace('\t', ' ')
                sentences = sent_tokenize(question['text'])
                for k, sent in enumerate(sentences):

                    for_10_pts = 'For 10 points, name '

                    if sent[:len(for_10_pts)] == for_10_pts:
                        sent = sent[len(for_10_pts):]
                    query_text = sent

                    title = question['page']
                    
                    out_l = query_text + '\t' + title + '\t' + gold_passage + '\n'
                    
                    ir_train_f.write(out_l)
                    
                    if k == len(sentences) - 1:
                        ir_train_last_sent_only_f.write(out_l)
                i += 1
                if i % 100 == 0:
                    print(i, 'of', len(questions))


112927 questions total
100 of 112927
200 of 112927
300 of 112927
400 of 112927
500 of 112927
600 of 112927
700 of 112927
800 of 112927
900 of 112927
1000 of 112927
1100 of 112927
1200 of 112927
1300 of 112927
1400 of 112927
1500 of 112927
1600 of 112927
1700 of 112927
1800 of 112927
1900 of 112927
2000 of 112927
2100 of 112927
2200 of 112927
2300 of 112927
2400 of 112927
Willy_Brandt has no wiki text
2500 of 112927
2600 of 112927
2700 of 112927
2800 of 112927
2900 of 112927
3000 of 112927
3100 of 112927
3200 of 112927
3300 of 112927
3400 of 112927
3500 of 112927
3600 of 112927
3700 of 112927
3800 of 112927
3900 of 112927
4000 of 112927
Willy_Brandt has no wiki text
4100 of 112927
4200 of 112927
School_for_Lovers has no wiki text
4300 of 112927
Tarquinia_(gens) has no wiki text
4400 of 112927
4500 of 112927
4600 of 112927
4700 of 112927
4800 of 112927
4900 of 112927
5000 of 112927
5100 of 112927
5200 of 112927
5300 of 112927
CRaSH has no wiki text
5400 of 112927
5500 of 112927
5600 of 1

48100 of 112927
48200 of 112927
48300 of 112927
48400 of 112927
48500 of 112927
48600 of 112927
48700 of 112927
48800 of 112927
48900 of 112927
49000 of 112927
49100 of 112927
49200 of 112927
49300 of 112927
49400 of 112927
49500 of 112927
49600 of 112927
49700 of 112927
49800 of 112927
49900 of 112927
50000 of 112927
50100 of 112927
50200 of 112927
50300 of 112927
50400 of 112927
50500 of 112927
50600 of 112927
50700 of 112927
50800 of 112927
50900 of 112927
51000 of 112927
51100 of 112927
51200 of 112927
51300 of 112927
51400 of 112927
51500 of 112927
51600 of 112927
51700 of 112927
51800 of 112927
51900 of 112927
52000 of 112927
52100 of 112927
52200 of 112927
52300 of 112927
52400 of 112927
52500 of 112927
52600 of 112927
52700 of 112927
52800 of 112927
52900 of 112927
53000 of 112927
53100 of 112927
53200 of 112927
53300 of 112927
53400 of 112927
53500 of 112927
53600 of 112927
53700 of 112927
53800 of 112927
53900 of 112927
54000 of 112927
54100 of 112927
54200 of 112927
54300 of

97300 of 112927
97400 of 112927
97500 of 112927
97600 of 112927
97700 of 112927
97800 of 112927
97900 of 112927
98000 of 112927
98100 of 112927
98200 of 112927
98300 of 112927
98400 of 112927
98500 of 112927
98600 of 112927
Hinduism has no wiki text
98700 of 112927
98800 of 112927
98900 of 112927
99000 of 112927
99100 of 112927
99200 of 112927
99300 of 112927
99400 of 112927
99500 of 112927
99600 of 112927
99700 of 112927
99800 of 112927
99900 of 112927
100000 of 112927
100100 of 112927
100200 of 112927
100300 of 112927
100400 of 112927
100500 of 112927
100600 of 112927
100700 of 112927
100800 of 112927
100900 of 112927
101000 of 112927
101100 of 112927
101200 of 112927
101300 of 112927
101400 of 112927
101500 of 112927
101600 of 112927
101700 of 112927
101800 of 112927
Hinduism has no wiki text
101900 of 112927
102000 of 112927
102100 of 112927
102200 of 112927
102300 of 112927
102400 of 112927
102500 of 112927
102600 of 112927
102700 of 112927
102800 of 112927
102900 of 112927
103000

In [3]:
## TRAINING

import torch
torch.cuda.empty_cache() 

# Configure the model
model_args = RetrievalArgs()
model_args.num_train_epochs = 5
model_args.output_dir = './retrieval_outputs_no_naming_qanta_train/'
model_args.train_batch_size = 12
model_args.eval_batch_size = 1
# model_args.hard_negatives = True
# model_args.overwrite_output_dir = True
# model_args.use_hf_datasets = True

model_type = "dpr"
context_encoder_name = "facebook/dpr-ctx_encoder-single-nq-base"
question_encoder_name = "facebook/dpr-question_encoder-single-nq-base"

model = RetrievalModel(
    model_type=model_type,
    context_encoder_name=context_encoder_name,
    query_encoder_name=question_encoder_name,
    args=model_args,
    embed_batch_size=16,
    retrieval_batch_size=512,
    retrieve_n_docs=5,
    use_cuda=True,
)

ir_train_tsv_path = '../data/ir_wiki_train_summaries_no_for_10_pts_qanta_train.tsv'
# ir_train_tsv_path = '../data/ir_wiki_train_summaries.tsv'
train_df = pd.read_csv(ir_train_tsv_path, sep='\t', error_bad_lines=False)
train_df = train_df.dropna(how='any',axis=0) 

queries = train_df['query_text'].tolist()

# wiki_summaries_path = '../data/wiki_summaries.txt'
# with open(wiki_summaries_path, encoding='utf-8') as f:
#     passages = f.read().splitlines()

model.train_model(train_df)

# hard_df = model.build_hard_negatives(
#     queries=queries,
#     passage_dataset=passages,
#     retrieve_n_docs=1,
# )

# train_df['hard_negative'] = hard_df['hard_negatives_0']

b'Skipping line 39876: expected 3 fields, saw 4\nSkipping line 55303: expected 3 fields, saw 4\nSkipping line 90696: expected 3 fields, saw 4\n'
b'Skipping line 471937: expected 3 fields, saw 4\nSkipping line 498211: expected 3 fields, saw 4\n'


  0%|          | 0/572936 [00:00<?, ?ex/s]

  0%|          | 0/573 [00:00<?, ?ba/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/47745 [00:00<?, ?it/s]

...

Running Epoch 1 of 5:   0%|          | 0/47745 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/47745 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/47745 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/47745 [00:00<?, ?it/s]

(238725, 0.21594138474818853)

In [ ]:
hard_df.head()
train_df['hard_negative'] = hard_df['hard_negatives_0']
print(len(train_df['hard_negative']), len(train_df['query_text']))
print(train_df.head())

train_df.to_csv('../data/ir_train_with_hard_negatives.tsv', sep='\t')

In [ ]:
torch.cuda.empty_cache() 

model_args = RetrievalArgs()
model_args.num_train_epochs = 5
model_args.output_dir = './retrieval_outputs_no_naming_qanta_train/'
model_args.train_batch_size = 12
model_args.eval_batch_size = 1
model_args.hard_negatives = True
# model_args.overwrite_output_dir = True
# model_args.use_hf_datasets = True

model_type = "dpr"
context_encoder_name = "facebook/dpr-ctx_encoder-single-nq-base"
question_encoder_name = "facebook/dpr-question_encoder-single-nq-base"

model = RetrievalModel(
    model_type=model_type,
    context_encoder_name=context_encoder_name,
    query_encoder_name=question_encoder_name,
    args=model_args,
    embed_batch_size=16,
    retrieval_batch_size=512,
    retrieve_n_docs=5,
    use_cuda=True,
)

model.train_model(train_df)

In [5]:
model2 = RetrievalModel(
    model_type='dpr',
    model_name='./retrieval_outputs_no_naming_qanta_train/',
    embed_batch_size=16,
    retrieval_batch_size=512,
)

In [29]:
wiki_summaries_path = '../data/wiki_summaries.txt'
with open(wiki_summaries_path, encoding='utf-8') as f:
    passages = f.read().splitlines()

to_predict = [
  "One work by this author uses printing, gunpowder, and the compass as symbols of personal ambition, national ambition, and the ambition of the human race to extend its grasp. This thinker described three forms of false learning as \"delicate\", \"contentious\", and \"fantastical\" in categorizing the \"distempers\" that impede academic progress. This thinker imagined a utopian university called Salomon's House, and he likened received systems of philosophy to stage plays that misrepresent the world, and thus labeled them \"idols of the theatre\". This author of The New Atlantis established the doctrine of inductive, empirical methodology. For 10 points, name this 17th-century English philosopher who wrote Novum Organum and spearheaded the Scientific Revolution.",
  "One work by this author uses printing, gunpowder, and the compass as symbols of personal ambition, national ambition, and the ambition of the human race to extend its grasp.",
  "This thinker described three forms of false learning as \"delicate\", \"contentious\", and \"fantastical\" in categorizing the \"distempers\" that impede academic progress.",
  "This thinker imagined a utopian university called Salomon's House, and he likened received systems of philosophy to stage plays that misrepresent the world, and thus labeled them \"idols of the theatre\".",
  "This author of The New Atlantis established the doctrine of inductive, empirical methodology.",
  "For 10 points, name this 17th-century English philosopher who wrote Novum Organum and spearheaded the Scientific Revolution.",
#   "For 10 points, name this play in which Orgon's family hosts the title religious hypocrite, a play by Moli",
]

predicted_passages, doc_ids, doc_vectors, doc_dicts = model2.predict(to_predict, prediction_passages=passages, retrieve_n_docs=10)
predicted_passages

Generating query embeddings: 0it [00:00, ?it/s]

Retrieving docs:   0%|          | 0/1 [00:00<?, ?it/s]

[['John Ruskin (8 February 1819 – 20 January 1900) was an English writer, philosopher and art critic of the Victorian era. He wrote on subjects as varied as geology, architecture, myth, ornithology, literature, education, botany and political economy. Ruskin\'s writing styles and literary forms were equally varied. He wrote essays and treatises, poetry and lectures, travel guides and manuals, letters and even a fairy tale. He also made detailed sketches and paintings of rocks, plants, birds, landscapes, architectural structures and ornamentation. The elaborate style that characterised his earliest writing on art gave way in time to plainer language designed to communicate his ideas more effectively. In all of his writing, he emphasised the connections between nature, art and society. Ruskin was hugely influential in the latter half of the 19th century and up to the First World War. After a period of relative decline, his reputation has steadily improved since the 1960s with the publica

In [15]:
doc_ids

array([[  328., 24592., 20749., ...,  5014., 35429.,  4191.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       ...,
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.]])

In [30]:
def summary_to_page(sql_cur, predicted_passages):
    a = []
    for answers_to_question in predicted_passages:
        b = []
        for answer in answers_to_question:
            row = cur.execute("SELECT page FROM wiki_text WHERE text_index = ?", (answer[:64],)).fetchone()
            if row:
                b.append(row[0])
        a.append(b)
    return a

summary_to_page(cur, predicted_passages)

[['John_Ruskin',
  'R._G._Collingwood',
  'Johann_Gottfried_Herder',
  'Samuel_Butler_(novelist)',
  'Charles_Augustin_Sainte-Beuve',
  'Walter_Benjamin',
  'Walter_Benjamin',
  'Giambattista_Vico',
  'Giambattista_Vico',
  'Lewis_Mumford'],
 ['Edward_Gibbon',
  'Edward_Gibbon',
  'Voltaire',
  'Voltaire',
  'Anthony_Ashley-Cooper,_3rd_Earl_of_Shaftesbury',
  'Anthony_Ashley-Cooper,_3rd_Earl_of_Shaftesbury',
  'John_Stuart_Mill',
  'Denis_Diderot',
  'Denis_Diderot',
  'Alexis_de_Tocqueville'],
 ['Jürgen_Habermas',
  'Georg_Simmel',
  'Georg_Simmel',
  'Max_Horkheimer',
  'Max_Weber',
  'Ferdinand_Tönnies',
  'Karl_Popper',
  'Karl_Popper',
  'John_Dewey',
  'John_Dewey'],
 ['Francis_Bacon',
  'Anthony_Ashley-Cooper,_3rd_Earl_of_Shaftesbury',
  'Anthony_Ashley-Cooper,_3rd_Earl_of_Shaftesbury',
  'John_Toland',
  'George_Berkeley',
  'Bernard_Mandeville',
  'Thomas_Hobbes',
  'Thomas_Hobbes',
  'William_Wollaston',
  'Johann_Gottfried_Herder'],
 ['Lewis_Mumford',
  'Lewis_Mumford',
  'A

In [8]:
import nltk
nltk.download('punkt')
def split_sents(text):
    sents = sent_tokenize(text)
    sents[-1] = sents[-1].replace("For 10 points, name ", "")
    sents[-1] = sents[-1].replace("For 10 points, ", "")

    sents.append(text)
    return sents

def predict(question):
    sents = split_sents(question)
    predicted_passages, doc_ids, doc_vectors, doc_dicts = model2.predict(sents, prediction_passages=passages, retrieve_n_docs=10)
    ans = summary_to_page(cur, predicted_passages)
    return ans

def answer_present(guesses, answer):
    return guesses.count(answer)

def index(lst, e):
    try:
        return lst.index(e)
    except ValueError:
        return -1

def answer_present_in_all(all_guesses, answer):
    times_present = [answer_present(guesses, answer) for guesses in all_guesses]
    index_presence = [index(guesses, answer) for guesses in all_guesses]
    return times_present, index_presence

def flatten(t):
    return [item for sublist in t for item in sublist]


pred = predict("For 10 points, name this play in which Orgon's family hosts the title religious hypocrite, a play by Moli")
answer_present_in_all(pred, 'The_Miser')
set(flatten(pred))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\patri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Generating query embeddings: 0it [00:00, ?it/s]

Retrieving docs:   0%|          | 0/1 [00:00<?, ?it/s]

{"Danton's_Death",
 'Le_Médecin_malgré_lui',
 'Tartuffe',
 'The_Balcony',
 'The_Chairs',
 'The_Misanthrope',
 'The_Miser'}

In [28]:
guessed_at_least_once = 0
total = 0
top_guesses = 0
k = 0
num_positive = 0
num_negative = 0
with open('../data/qanta_dev_buzzer_train.csv', 'w', encoding='utf-8') as out_f:
    
    out_f.write("sentence,first_idx,occurrences,correct\n")
    
    with open('../data/qanta_dev.json') as f:
        questions = json.load(f)['questions']
        for question in questions:
            answer = question['page']
            guesses = predict(question['text'])
            times_present, idx_presence = answer_present_in_all(guesses, answer)
            top_guess = idx_presence[-2]
            print(top_guess, answer)
            
#             set_of_guesses = set(flatten(guesses))

#             for guess in set_of_guesses:
                

#                 correct = 1 if answer == guess else 0
                
#                 if correct == 1 or num_negative * 1.5 < num_positive:
                    
#                     times_present, idx_presence = answer_present_in_all(guesses, guess)

#                     for i, (first_idx, occurences) in enumerate(zip(idx_presence[:-1], times_present[:-1])):
#                         out_f.write(f"{i+1},{first_idx},{occurences},{correct}\n")
#                     out_f.write(f"{0},{idx_presence[-1]},{times_present[-1]},{correct}\n")
                    
#                     if correct == 1:
#                         num_positive += 1
#                     else:
#                         num_negative += 1


            if sum(times_present) > 0:
                guessed_at_least_once += 1
            if top_guess == 0:
                top_guesses += 1
            total += 1

            print('Times guessed at least once acc:', guessed_at_least_once / total)
            print('Top acc guess:', top_guesses / total)

            print(times_present)
            print(idx_presence)

Generating query embeddings: 0it [00:00, ?it/s]

Retrieving docs:   0%|          | 0/1 [00:00<?, ?it/s]

-1 Texas_annexation
Times guessed at least once acc: 1.0
Top acc guess: 0.0
[0, 0, 0, 1, 0, 0, 1, 0, 1]
[-1, -1, -1, 7, -1, -1, 6, -1, 2]


Generating query embeddings: 0it [00:00, ?it/s]

Retrieving docs:   0%|          | 0/1 [00:00<?, ?it/s]

...

Generating query embeddings: 0it [00:00, ?it/s]

Retrieving docs:   0%|          | 0/1 [00:00<?, ?it/s]

0 Japan
Times guessed at least once acc: 0.7314343845371313
Top acc guess: 0.23499491353001017
[1, 0, 1, 1, 1, 1, 1]
[7, -1, 2, 0, 0, 0, 0]


Generating query embeddings: 0it [00:00, ?it/s]

KeyboardInterrupt: 

In [30]:
df = pd.read_csv('../data/qanta_test_buzzer_train.csv')
cdf = df[df['correct'] == 1]
print(len(df))
print(len(cdf))

999230
22773


In [20]:
def split_sents2(text):
    sents = sent_tokenize(text)
#     sents[-1] = sents[-1].replace("For 10 points, name ", "")
#     sents[-1] = sents[-1].replace("For 10 points, ", "")
    a = ""
    for s in sents:
        a += s
        sents.append(a)

#     sents.append(text)
    return sents

def predict_add(question):
    sents = split_sents2(question)
    predicted_passages, doc_ids, doc_vectors, doc_dicts = model2.predict(sents, prediction_passages=passages, retrieve_n_docs=10)
    ans = summary_to_page(cur, predicted_passages)
    return ans

with open('../data/qanta_dev.json') as f:
        questions = json.load(f)['questions']
        for question in questions:
            answer = question['page']
            guesses = predict(question['text'])
            print(question['text'])
            for g in guesses:
                print(g)
            break

Generating query embeddings: 0it [00:00, ?it/s]

Retrieving docs:   0%|          | 0/1 [00:00<?, ?it/s]

Robert Walker argued that failing to take this action would lead to an overflow of Northern insane asylums and British intervention. Juan Almonte resigned his diplomatic post in indignation over this event. Isaac Van Zandt began discussing this plan with Abel Upshur before Upshur died. Anson Jones proposed this plan, which reduced his own power but reserved the weaker side the right to split into five parts in the future. Five years after it, a payment of 10 million dollars helped the area at issue repay debts. The Regulator-Moderator war was calmed just before this deal was struck. The question of whether the Nueces River became a southern border in this transaction led to war later in James K. Polk's presidency. For 10 points, name this deal that ended the independent Lone Star Republic.
['Indian_removal', 'Indian_removal', 'Continental_System', 'Trail_of_Tears', 'Emancipation_reform_of_1861', 'Expulsion_of_the_Acadians', 'Expulsion_of_the_Acadians', 'Gadsden_Purchase', 'Gadsden_Purc